In [ ]:
import pandas as pd
import requests 
import  matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import numpy as np
import re

In [ ]:
MMS_STATS_URL = "https://www.mixedmartialarts.com/"
UFC_STATS_URL = "https://www.ufc.com/athlete/"
UFC_UPCOMING_EVENTS = "https://en.wikipedia.org/wiki/List_of_UFC_events"

class FighterBot:

    def __init__(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.df_0 = pd.DataFrame()
        self.df_1 = pd.DataFrame()
        self.fighters = []
        self.fighter_errors = []
        self.fighter_db = pd.read_csv('./fighters.csv')
        self.fight_card = {}

    def closeBrowser(self):
        self.driver.close()
        
    def get_fighter_failures(self):
        return self.fighter_errors
    
    def remove_extra_info(self, name):
        return re.sub(' \([^)]*\)','', name)
    
    def prepare_fighter_bot(self):
        fights = self.get_next_ufc_fight()
        self.fighters = fights["Fighter_1"].tolist() + fights["Fighter_2"].tolist()

        for fighter in self.fighters:
            if(fighter not in self.fighter_db['name'].tolist()):
                if(self.df_0.empty):
                    self.df_0 = self.get_fighter_stats(fighter)
                else:
                    self.df_0 = self.df_0.append(self.get_fighter_stats(fighter))
        
        self.closeBrowser()
        
        
        driver_2 = webdriver.Chrome(ChromeDriverManager().install())
        if(len(self.get_fighter_failures()) > 0):
            f_dict = {}
            for i,fights in enumerate(self.fight_card['fights']):
                fight_e = [fights.index(e) for e in errors if e in fights]
                f_dict[i] = fight_e

            for k,v in f_dict.items():
                if(len(v) > 0):
                    opponent = self.get_opponent(k, v)
                    print(opponent)
                    self.df_0 = self.df_0.append(self.find_fighter_via_opponent(driver_2, opponent))
            
        self.write_to_csv('./fighters.csv', self.df_0)
        self.closeBrowser()
        
        return self.df_0 
    
    def get_opponent(self, k, v):
        idx = [val for val in v]
        if(idx == 0):
            return self.fight_card['fights'][k][0]
        else:
            return self.fight_card['fights'][k][1]
        
    def find_fighter_via_opponent(self, driver_2, opponent):
        driver_2 = self.search_for_fighter_mma_stats(opponent)
        
        try:
            name = driver_2.find_element_by_xpath("//*[@id='DataTables_Table_0']/tbody/tr[1]/td[4]/a").text
            f_url = driver_2.find_element_by_xpath("//*[@id='DataTables_Table_0']/tbody/tr[1]/td[4]/a[@href]")
            url = f_name.get_attribute('href')
        except NoSuchElementException:
            return
        
        df = self.get_fighter_stats(name, True, url)
        return df

    def write_to_csv(self, fname, df):
        df.to_csv(fname, mode='a', header=False)
    
    def generate_fight_card_dict(self, fight_name, df):
        card = df[["Fighter_1", "Fighter_2"]].to_dict('split')
        self.fight_card['fight_card'] = fight_name
        self.fight_card['fights'] = card['data']
    
    def get_next_ufc_fight(self):
        driver = self.driver
        driver.get(UFC_UPCOMING_EVENTS)
        time.sleep(1)
        fight_name = driver.find_element_by_id('firstHeading').text
        fights_table = driver.find_element_by_xpath("//*[@id='Scheduled_events']/tbody")
        fights = fights_table.find_elements_by_tag_name("tr")
        upcoming_fights = lambda: driver.find_element_by_xpath("//*[@id='Scheduled_events']/tbody/tr["+str(len(fights))+"]/td[1]/a")
        upcoming_fights().click()
        time.sleep(4)
        
        dfs = pd.read_html(driver.current_url)
        df = dfs[2]
        #self.closeBrowser()
        if([2,4,5,6,7] in df.columns.tolist()):
            df.drop(columns=[2,4,5,6,7], inplace=True)
            
        df.rename(columns={0: 'class', 1 : 'Fighter_1', 
                  3: 'Fighter_2'}, inplace=True)
        
        df.dropna(subset=['Fighter_1','Fighter_2'], inplace=True)
        
        df["Fighter_1"] = df["Fighter_1"].apply(lambda name: self.remove_extra_info(name))
        df["Fighter_2"] = df["Fighter_2"].apply(lambda name: self.remove_extra_info(name))
        
        self.generate_fight_card_dict(fight_name, df)
        
        return df
        

    def get_fighter_stats(self, name, hardcoded=False, url=None):
        driver = self.search_for_fighter_mma_stats(name)
        fighter_stat_url = driver.current_url
        
        if(hardcoded == True):
            fighter_stat_url = driver.get(url)
            
        if("search" in fighter_stat_url):
            self.fighter_errors.append(name)
            return pd.DataFrame()
        elif("error=nofighter" in fighter_stat_url):
            self.fighter_errors.append(name)
            return pd.DataFrame()
        else:
            dfs = pd.read_html(fighter_stat_url)
            stats_df = self.clean_up_stat_data(dfs, name)
            ufc_stats = self.search_for_fighter_ufc_stats(name)
            stats_df = stats_df.merge(ufc_stats, on='name', how='left')
            return stats_df
            
    
    def get_fighter_history(self, dfs):
        return dfs[7].dropna(axis=1, inplace=True)
    
    def search_for_fighter_mma_stats(self, name):
        driver = self.driver
        driver.get(MMS_STATS_URL)
        time.sleep(1)
        try:
            fighter_search = driver.find_element_by_xpath("//*[@id='header']/div/nav[2]/div/div/form/div/input")
            fighter_search.clear()
            fighter_search.send_keys(name)
            fighter_search.send_keys(Keys.RETURN)
            return driver
        except NoSuchElementException:
            return pd.DataFrame()    
    
    def search_for_fighter_ufc_stats(self, name):
        ufc_stats = {}
        driver = self.driver
        names = name.split()
        driver.get(UFC_STATS_URL + "-".join(names))
                                                          
        try:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 4);")
            time.sleep(3)

            striking_stats = self.get_striking_accuracy(driver)
            grappling_stats = self.get_grappling_accuracy(driver)
            ufc_stats_fighter = self.get_fighter_ufc_stats(driver)
            
            ufc_stats.update(striking_stats)
            ufc_stats.update(grappling_stats)
            ufc_stats.update(ufc_stats_fighter)
            ufc_stats.update({'name': name})
            
            self.df_1 = pd.DataFrame(ufc_stats, index=[ufc_stats['name']])
            
            #self.closeBrowser()
            
            return self.df_1
            
        except NoSuchElementException:
            self.fighter_errors.append(name)
            return pd.DataFrame()  
        
        
        
    def get_fighter_ufc_stats(self, driver):
        ufc_stats = {}
        time.sleep(3)
        try:
            comment_button = lambda: self.driver.find_element_by_xpath("//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[2]/button/span")
            comment_button().click()
        except NoSuchElementException:
            ufc_stats['tot_strikes_landed_per_min'] = 0
            ufc_stats['tot_strikes_attempted_per_min'] = 0
            ufc_stats['sig_strike_def_%'] = 0
            ufc_stats['takedown_defense_%'] = 0
            ufc_stats['avg_fight_time'] = 0
        
        time.sleep(2)
        
        ufc_stats['tot_strikes_landed_per_min'] = self.get_value_by_xpath(driver, ufc_stats, 'tot_strikes_landed_per_min', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[1]/div[1]/div[1]/div/div[1]/div[1]")
        ufc_stats['tot_strikes_attempted_per_min'] = self.get_value_by_xpath(driver, ufc_stats, 'tot_strikes_attempted_per_min', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[1]/div[2]/div[1]/div/div[1]/div[1]")
        ufc_stats['sig_strike_def_%'] = self.get_value_by_xpath(driver, ufc_stats, 'sig_strike_def_%', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[1]/div[2]/div[1]/div/div[1]/div[1]")
        ufc_stats['takedown_defense_%'] = self.get_value_by_xpath(driver, ufc_stats, 'takedown_defense_%', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[1]/div[2]/div[1]/div/div[3]/div[1]")
        ufc_stats['avg_fight_time'] = self.get_value_by_xpath(driver, ufc_stats, 'avg_fight_time', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[6]/div[1]/div[2]/div[2]/div/div[3]/div[1]")

        
        return ufc_stats
        
    def get_striking_accuracy(self, driver):
        striking_stats = {}
        striking_stats['strikes_landed'] = self.get_value_by_xpath(driver, striking_stats, 'strikes_landed', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[5]/div/section/div[1]/div/div[2]/dl/dd[1]")
        striking_stats['strikes_attempted'] = self.get_value_by_xpath(driver, striking_stats, 'strikes_attempted', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[5]/div/section/div[1]/div/div[2]/dl/dd[2]")
        
        return striking_stats
    
    def get_grappling_accuracy(self, driver):
        grappling_stats = {}
        grappling_stats['tds_landed'] = self.get_value_by_xpath(driver, grappling_stats, 'tds_landed', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[5]/div/section/div[2]/div/div[2]/dl/dd[1]")
        grappling_stats['tds_attempted'] = self.get_value_by_xpath(driver, grappling_stats, 'tds_attempted', "//*[@id='block-mainpagecontent']/div/div/div[2]/div[5]/div/section/div[2]/div/div[2]/dl/dd[2]")
            
        return grappling_stats
    
    def get_value_by_xpath(self, driver, obj, key, xpath):
        try:
            obj[key] = driver.find_element_by_xpath(xpath).text
        except:
            obj[key] = 0
        return obj[key]
        
    def reformat_df(self, df, name):
        df = pd.DataFrame(df.T)
        df.columns = df.iloc[0]
        df = df.reindex(df.index.drop(0))
        df.dropna(axis=1, inplace=True)
        df["name"] = name
        df.set_index('name', inplace=True)
        return df
    
    def clean_up_stat_data(self, dfs, name): 
        df_0 = self.reformat_df(dfs[0],name)
        df_2 = self.reformat_df(dfs[2],name)
        df_4 = self.reformat_df(dfs[4],name)
        df_6 = self.reformat_df(dfs[6],name)
        df_0 = df_0.merge(df_2, on='name',how='left')
        df_0 = df_0.merge(df_4, on='name',how='left')
        df_0 = df_0.merge(df_6, on='name', how='left')
        
        return df_0
    
    def merge_dataframe(self, df):
        return self.df_0.merge(df, on='name',how='left')

In [ ]:
fighter_stats = {}
fb = FighterBot()
df_new = fb.prepare_fighter_bot()
#df_2 = fb.search_for_fighter_ufc_stats("Frankie Edgar")

In [ ]:
df_new.set_index('name', inplace=True)

In [ ]:
df_new.columns

In [ ]:
df_new.to_csv('./fighters.csv')

In [ ]:
f = pd.read_csv('./fighters.csv')

In [ ]:
fb.getFighterFailures()

In [ ]:
#https://en.wikipedia.org/wiki/UFC_on_ESPN:_Covington_vs._Lawler

In [ ]:
fb_2 = FighterBot()
df_fights = fb_2.get_next_ufc_fight()

In [ ]:
errors = fb.getFighterFailures()

In [ ]:
f_dict = {}
for i,fights in enumerate(df_fights_dict['data']):
    fight_e = [fights.index(e) for e in errors if e in fights]
    f_dict[i] = fight_e

        

In [ ]:
for k,v in f_dict.items():
    if(len(v) > 0):
        opponent = get_opponent(k, v)
        print(opponent)

In [ ]:
def get_opponent(k,v):
    idx = [val for val in v]
    if(idx == 0):
        return df_fights_dict['data'][k][0]
    else:
        return df_fights_dict['data'][k][1]

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.mixedmartialarts.com/fighter/Clay-Guida:2653CA1D2BE858FF")
time.sleep(3)
f_name = driver.find_element_by_xpath("//*[@id='DataTables_Table_0']/tbody/tr[1]/td[4]/a[@href]")
url = f_name.get_attribute('href')
driver.get(url)
driver.close()

### METRICS FOR FIGHTERS

* Wrestling
* Aggression
* Striking
* Confidence